In [ ]:
import warnings
warnings.simplefilter('ignore')

import gc
import re
from collections import defaultdict, Counter

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
from tqdm.auto import tqdm

In [ ]:
df_sess = pd.read_csv('sessions_train.csv')
df_prod = pd.read_csv('products_train.csv')

In [29]:
# df_sess['locale'].unique()
df_sess.columns

Index(['prev_items', 'next_item', 'locale'], dtype='object')

In [28]:
df_prod_task1 = df_prod[df_prod['locale'].isin(['UK', 'DE', 'JP'])]
df_prod_task1

,id,locale,title,price,brand,color,size,model,material,author,desc
0,B005ZSSN10,DE,RED DRAGON Amberjack 3 - Steel Tip 22 Gramm Wo...,30.95,RED DRAGON,NaN,NaN,RDD0089,NaN,NaN,Amberjacks Steel Dartpfeile sind verfügbar in ...
1,B08PRYN6LD,DE,Simply Keto Lower Carb* Schokodrops ohne Zucke...,17.90,Simply Keto,NaN,750 g (1er Pack),NaN,NaN,NaN,🌱 NATÜRLICHE SÜSSE DURCH ERYTHRIT - Wir stelle...
2,B09MBZJ48V,DE,"Sennheiser 508377 PC 5.2 Chat, Stilvolles Mult...",68.89,Sennheiser,Multi-Colour,One size,508377,Kunstleder,NaN,3.5 MM BUCHSE - Kann problemlos an Geräte mit ...
3,B08ZN6F26S,DE,AmyBenton Auto ab 1 2 3 ahre - Baby Aufziehbar...,18.99,Amy & Benton,Animal Car,NaN,2008B,aufziehauto 1 jahr,NaN,【Auto aufziehbar】: Drücken Sie einfach leicht ...
4,B094DGRV7D,DE,PLAYMOBIL - 70522 - Cavaliere mit grauem Pony,7.17,PLAYMOBIL,Nicht Zutreffend.,OneSize,70522,Polypropylen,NaN,Inhalt: 1 Stück
...,...,...,...,...,...,...,...,...,...,...,...
1413511,B08D7KW8VK,UK,TOMHOUSEE Anime Cosplay Short Straight Hair Wi...,9.99,TOMHOUSEE,Deep Grey Yuki,NaN,NaN,Synthetic,NaN,NaN
1413512,B073WXLXR9,UK,Crystals NEW brilliant ink twister bingo dabbe...,8.99,CRYSTALS,"Orange,blue,green,pink,red,purple",NaN,NaN,Plastic,NaN,NaN
1413513,1529393833,UK,"Before I Do: the new, funny and unexpected lov...",4.50,Hodder Paperbacks,NaN,NaN,NaN,NaN,"Cousens, Sophie",NaN
1413514,B0B3TJ1NDN,UK,"Black iPhone Charger Cable, iPhone Charger Bra...",4.49,AA-TECH,Black,2M,brd-ip-black-2022,Nylon Braided,NaN,Added Protection: An additional layer of prote...


In [26]:
df_sess_task1 = df_sess[df_sess['locale'].isin(['UK', 'DE', 'JP'])]
df_sess_task1
# these can be decreased to smaller datasets to try faster

,prev_items,next_item,locale
0,['B09W9FND7K' 'B09JSPLN1M'],B09M7GY217,DE
1,['B076THCGSG' 'B007MO8IME' 'B08MF65MLV' 'B001B...,B001B4THSA,DE
2,['B0B1LGXWDS' 'B00AZYORS2' 'B0B1LGXWDS' 'B00AZ...,B0767DTG2Q,DE
3,['B09XMTWDVT' 'B0B4MZZ8MB' 'B0B7HZ2GWX' 'B09XM...,B0B4R9NN4B,DE
4,['B09Y5CSL3T' 'B09Y5DPTXN' 'B09FKD61R8'],B0BGVBKWGZ,DE
...,...,...,...
3272711,['B06XK89969' 'B01NGT5NF4' 'B00D5Z89C8' 'B07ZV...,B07VL2W1DR,UK
3272712,['B076M85W1K' 'B07L8792Q9' 'B095RW318L' 'B095R...,B095RQ2LCY,UK
3272713,['B00JQDIQRQ' 'B001O59QQE'],B088M5YT6Y,UK
3272714,['B07QMHMLJZ' 'B07FPYYMC4'],B07PFF814D,UK


In [ ]:
# df_test = pd.read_csv('sessions_test_task1.csv')
# df_test

In [25]:
def str2list(x):
    x = x.replace('[', '').replace(']', '').replace("'", '').replace('\n', ' ').replace('\r', ' ')
    l = [i for i in x.split() if i]
    return l

In [70]:
import numpy as np
import pandas as pd
from math import log10
import collections as col

class AssociationRules: 
    '''
    SequentialRules(steps = 3, weighting='div', pruning=0.0)
        
    Parameters
    --------
    steps : int
        TODO. (Default value: 3)
    weighting : string
        TODO. (Default value: 3)
    pruning : float
        TODO. (Default value: 20)
    
    '''
    
    def __init__( self, pruning=20, session_key='SessionId', item_key='ItemId' ):
        self.pruning = pruning
        self.session_key = session_key
        self.item_key = item_key
        self.session = -1
        self.session_items = []
            
    def fit( self, data, test=None ):
        '''
        Trains the predictor.
        
        Parameters
        --------
        data: pandas.DataFrame
            Training data. It contains the transactions of the sessions. It has one column for session IDs, one for item IDs and one for the timestamp of the events (unix timestamps).
            It must have a header. Column names are arbitrary, but must correspond to the ones you set during the initialization of the network (session_key, item_key, time_key properties).
        
            
        '''

        cur_session = -1
        last_items = []
        rules = dict()
        
        index_session = data.columns.get_loc( self.session_key )
        index_item = data.columns.get_loc( self.item_key )
        
        for row in data.itertuples( index=False ):
            
            session_id, item_id = row[index_session], row[index_item]
            
            if session_id != cur_session:
                cur_session = session_id
                last_items = []
            else: 
                for item_id2 in last_items: 
                    
                    if not item_id in rules :
                        rules[item_id] = dict()
                    
                    if not item_id2 in rules :
                        rules[item_id2] = dict()
                    
                    if not item_id in rules[item_id2]:
                        rules[item_id2][item_id] = 0
                    
                    if not item_id2 in rules[item_id]:
                        rules[item_id][item_id2] = 0
                    
                    rules[item_id][item_id2] += 1
                    rules[item_id2][item_id] += 1
                    
            last_items.append( item_id )
        
        if self.pruning > 0 :
            self.prune( rules )
            
        self.rules = rules
    
    def linear(self, i):
        return 1 - (0.1*i) if i <= 10 else 0
    
    def same(self, i):
        return 1
    
    def div(self, i):
        return 1/i
    
    def log(self, i):
        return 1/(log10(i+1.7))
    
    def quadratic(self, i):
        return 1/(i*i)
    
    def predict_next(self, session_id, input_item_id, predict_for_item_ids, skip=False, mode_type='view', timestamp=0):
        '''
        Gives predicton scores for a selected set of items on how likely they be the next item in the session.
                
        Parameters
        --------
        session_id : int or string
            The session IDs of the event.
        input_item_id : int or string
            The item ID of the event.
        predict_for_item_ids : 1D array
            IDs of items for which the network should give prediction scores. Every ID must be in the set of item IDs of the training set.
            
        Returns
        --------
        out : pandas.Series
            Prediction scores for selected items on how likely to be the next item of this session. Indexed by the item IDs.
        
        '''
        if session_id != self.session:
            self.session_items = []
            self.session = session_id
        
        if mode_type == 'view':
            self.session_items.append( input_item_id )
            
        if skip:
            return
        
        preds = np.zeros( len(predict_for_item_ids) ) 
             
        if input_item_id in self.rules:
            for key in self.rules[input_item_id]:
                preds[ predict_for_item_ids == key ] = self.rules[input_item_id][key]
        
        series = pd.Series(data=preds, index=predict_for_item_ids)
        
        series = series / series.max()
        
        return series 
    
    def prune(self, rules): 
        '''
        Gives predicton scores for a selected set of items on how likely they be the next item in the session.
        Parameters
            --------
            rules : dict of dicts
                The rules mined from the training data
        '''
        for k1 in rules:
            tmp = rules[k1]
            if self.pruning < 1:
                keep = len(tmp) - int( len(tmp) * self.pruning )
            elif self.pruning >= 1:
                keep = self.pruning
            counter = col.Counter( tmp )
            rules[k1] = dict()
            for k2, v in counter.most_common( keep ):
                rules[k1][k2] = v              
                
    def clear(self):
        self.rules = {}

    def support_users(self):
        '''
          whether it is a session-based or session-aware algorithm
          (if returns True, method "predict_with_training_data" must be defined as well)

          Parameters
          --------

          Returns
          --------
          True : if it is session-aware
          False : if it is session-based
        '''
        return False


In [68]:
# Split the data
train_data = df_sess_task1['prev_items'][:50000]
test_data = df_sess_task1['prev_items'][50000:100000]

train_data

0                              ['B09W9FND7K' 'B09JSPLN1M']
1        ['B076THCGSG' 'B007MO8IME' 'B08MF65MLV' 'B001B...
2        ['B0B1LGXWDS' 'B00AZYORS2' 'B0B1LGXWDS' 'B00AZ...
3        ['B09XMTWDVT' 'B0B4MZZ8MB' 'B0B7HZ2GWX' 'B09XM...
4                 ['B09Y5CSL3T' 'B09Y5DPTXN' 'B09FKD61R8']
                               ...                        
49995                          ['B09SNZWHSK' 'B071GFD83G']
49996    ['B0B8BZK7XP' 'B0B8BQWNBJ' 'B0B81ZWGRM' 'B0B81...
49997    ['B09XXRVK57' 'B09XXT9W7Q' 'B0B253MDJ1' 'B0B24...
49998    ['B09JRJ3NYV' '3747203949' 'B09JRLJX7G' 'B08LJ...
49999    ['3741524794' '3473315788' '3473316466' '34734...
Name: prev_items, Length: 50000, dtype: object

In [79]:
# # Fit an AR model
# model = AutoReg(train_data,lags=4)
# ar_model = model.fit()

# # Evaluate the model
# predictions = ar_model.predict(start=len(train_data), end=len(train_data)+len(test_data)-1)
ar = AssociationRules()
ar.fit(data=df_sess_task1)


KeyError: 'SessionId'

In [ ]:
next_item_dict = defaultdict(list)

for _, row in tqdm(df_sess.iterrows(), total=len(df_sess)):
    prev_items = str2list(row['prev_items'])
    next_item = row['next_item']
    prev_items_length = len(prev_items)
    if prev_items_length <= 1:
        next_item_dict[prev_items[0]].append(next_item)
    else:
        for i, item in enumerate(prev_items[:-1]):
            next_item_dict[item].append(prev_items[i+1])
        next_item_dict[prev_items[-1]].append(next_item)

In [ ]:
for _, row in tqdm(df_test.iterrows(), total=len(df_test)):
    prev_items = str2list(row['prev_items'])
    prev_items_length = len(prev_items)
    if prev_items_length <= 1:
        continue
    else:
        for i, item in enumerate(prev_items[:-1]):
            next_item_dict[item].append(prev_items[i+1])

In [ ]:
next_item_map = {}

for item in tqdm(next_item_dict):
    counter = Counter(next_item_dict[item])
    next_item_map[item] = [i[0] for i in counter.most_common(100)]

In [ ]:
k = []
v = []

for item in next_item_dict:
    k.append(item)
    v.append(next_item_dict[item])
    
df_next = pd.DataFrame({'item': k, 'next_item': v})
df_next = df_next.explode('next_item').reset_index(drop=True)
df_next

In [ ]:
top200 = df_next['next_item'].value_counts().index.tolist()[:200]

In [ ]:
df_test['last_item'] = df_test['prev_items'].apply(lambda x: str2list(x)[-1])
df_test['next_item_prediction'] = df_test['last_item'].map(next_item_map)
df_test

In [ ]:
preds = []

for _, row in tqdm(df_test.iterrows(), total=len(df_test)):
    pred_orig = row['next_item_prediction']
    pred = pred_orig
    prev_items = str2list(row['prev_items'])
    if type(pred) == float:
        pred = top200[:100]
    else:
        if len(pred_orig) < 100:
            for i in top200:
                if i not in pred_orig and i not in prev_items:
                    pred.append(i)
                if len(pred) >= 100:
                    break
        else:
            pred = pred[:100]
    preds.append(pred)

In [ ]:
# df_test['next_item_prediction'] = preds
# df_test

In [ ]:
# df_test['next_item_prediction'].apply(len).describe()

In [ ]:
# df_test[['locale', 'next_item_prediction']].to_parquet('submission_task1.parquet', engine='pyarrow')

In [ ]:
# print(df_sess["next_item"])
# print()